<a href="https://colab.research.google.com/github/Parisman1/CSCE4205_Audio_ML/blob/main/Project_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#This is for the fun part